# Część druga
<div class="alert alert-block alert-danger">
<b>Uwaga:</b> Należy upewnić się że w prawym górnym rogu tego okna wybrany jest kernel "omics-genomika" a nie "Python 3"
</div>

# Wykrywanie wariantów DNA
<img src="png/vd-image1.png">

## Ustawienia zmiennych

In [ ]:
import os

workspace = "/home/jovyan/work/git/edugen_pub/"
os.environ['WORKSPACE'] = workspace
! echo ${WORKSPACE}

edugen_bucket  = "gs://biodatageeks/sequila/edugen/"
os.environ['EDUGEN_BUCKET'] = edugen_bucket

## Wykrywanie wariantów GATK HaplotypeCaller w trybie standardowym

GATK HaplotypeCaller jest jednym z najbardziej popularnych narzędzi do wykrywania wariantów germinalnych w danych pochodzących z sekwencjonowania wysokoprzepustowego. Dokumentacja znajduje się [pod linkiem](https://gatk.broadinstitute.org/hc/en-us/articles/360037225632-HaplotypeCaller). HaplotypeCaller jest częścią paczki programów Genome Analysis ToolKit (GATK) stworzonych przez Broad Institute. Oferuje on również m.in. algorytm wykrywania wariantów zoptymalizowany do wykrywania wariantów somatycznych - [Mutect2](https://gatk.broadinstitute.org/hc/en-us/articles/360037593851-Mutect2).

Wywolanie GATK HC w najprostszej wersji dla pojedynczej próbki

In [ ]:
! gatk HaplotypeCaller \
    -R ${WORKSPACE}/ref/ref.fasta \
    -I ${WORKSPACE}/bam/mother.bam \
    -O ${WORKSPACE}/sandbox/motherHC.vcf \
    -L 20:10,000,000-10,200,000

## Wizualizacja BAM i VCF w programie IGV

Najczęciej do wizualizacji danych z uliniowienia (i nie tylko) wykorzystuje się narzedzie IGV - [Integrative Genome Viewer](https://igv.org/doc/desktop/#DownloadPage/). Program ten instalowany jest na komputerze, natomiast na zajęciach skorzystamy z paczki [igv-notebook](https://github.com/igvteam/igv-notebook) która jest pythonową "owijką" dla programu igv.js, dlatego też możliwe do użycia opcje i argumenty znajdziemy w [dokumnetacji tego programu](https://igv.org/doc/igvjs/#tracks/Tracks/).

### Uruchomienie IGV

In [ ]:
import igv_notebook
igv_notebook.init()
igv_browser= igv_notebook.Browser(
    {
        "genome": "hg19",
        "locus": "chr20:10002294-10002623",
        "tracks": [{
            "name": "BAM",
            "url": edugen_bucket + "bam/mother.bam",
            "indexURL": edugen_bucket + "bam/mother.bai",
            "format": "bam",
            "type": "alignment"
        }]
    }
)

In [ ]:
! zcat ${WORKSPACE}/sandbox/motherHC.vcf.gz | grep 100024

<div class="alert alert-block alert-warning">
<b>Zadanie 2:</b> Jaka jest zygotyczność dwóch widocznych wariantów? Czy jest ona zgodna z tym co widzimy w odczytach ? 
</div>

<div class="alert alert-block alert-warning">
<b>Zadanie 3:</b> Kliknij na ikonę edycji track'u z odczytami i zaznacz opcję "Soft clip reads", co widzimy w odczytach?
</div>

Algorytm mapowania (BWA MEM) zdecydował, że kara za obcięcie odczytów (Soft clipping) będzie mniejsza niż wstawienie insersji/delecji.

Do oceny wiarygodności wariantów wykorzystywany jest "Phread-scaled Likelihoods of Genotypes", dokładne omówienie można znaleźć [pod linkiem](https://gatk.broadinstitute.org/hc/en-us/articles/360035890451-Calculation-of-PL-and-GQ-by-HaplotypeCaller-and-GenotypeGVCFs).

## Ponowne uliniowienie odczytów i zasemblowane haplotypy

HaplotypeCaller posiada parametr -bamout pozwalający na wyświetlenie ponownie uliniowionych odczytów,
które następnie są wykorzystywane do wykrywania wariantów.


In [ ]:
! gatk HaplotypeCaller \
    -R ${WORKSPACE}/ref/ref.fasta \
    -I ${WORKSPACE}/bam/mother.bam\
    -O ${WORKSPACE}/sandbox/motherHCdebug.vcf \
    -bamout ${WORKSPACE}/sandbox/motherHCdebug.bam \
    -L 20:10,002,000-10,003,000

<div class="alert alert-block alert-warning">
<b>Zadanie 4*:</b> Wyświetl dla porównania obydwa BAMy (zwykły i debug) oraz VCF. Przy wyświetlaniu BAM'ów ustaw opcję 
    showSoftClips = True, aby od razu wyświetlały się split ready. Inne opcje są dostępne na: https://github.com/igvteam/igv.js/wiki/Alignment-Track
</div>

Ponieważ jesteśmy zainteresowani jednym miejscem (20:10,002,000-10,003,000) to dokonaliśmy reuliniowienia tylko tego fragmentu. 

Po ponownym uliniowieniu zniknęły obcięte odczyty (soft clipped reads) . HaplotypeCaller wykorzystuje odczyty 'soft-clipped' do ponownego uliniowienia. Z analizy odczytów wynika, że insercja i pobliski SNP są ze sobą sfazowane. Wynika z tego, że HaplotypeCaller znalazł różne możliwe uliniowienia po wykonaniu lokalnej reasemblacji. Po reasemblacji występuje wystarczająco dużo odczytów wskazujących na występowanie insercji, która byłaby pominięta w przypadku użycia prostszych algorytmów, nie wykorzystujących reasemblacji (np. UnifiedGenotyper)

<div class="alert alert-block alert-warning">
<b>Zadanie 5:</b> Przyjrzyj się insercji w poszczególnych odczytach.  Czy wszędzie jest ona taka sama?
</div>

<div class="alert alert-block alert-warning">
<b>Zadanie 6*:</b> W jaki sposób wyświetlić w IGV haplotypy utworzone przez HC?
</div>

Czerwone odczyty nie są prawdziwe - (spójrz na RG). Są to odczyty reprezentujące haplotypy utworzone przez HaplotypeCaller. Co jeżeli pokolorujemy po tagu HC?
<div class="alert alert-block alert-warning">
<b>Zadanie 7:</b> Podejrzyj wsparcie dla każdego haplotypu poprzez pokolorowanie po tag'u HC zamiast RG. HaplotypeCaller oznacza odpowiednim tagiem HC wszystkie odczyty jednoznacznie wskazujący na dany "sztuczny haplotyp". Co teraz sądzisz o decyzji Variant Caller'a w sprawie wyboru typu insercji?
</div>

<div class="alert alert-block alert-warning">
<b>Zadanie 8:</b> Oddal widok i zobacz ile różnych haplotypów zostało znalezionych dla kolejnych trzech spójnych regionów. Dlaczego w regionie po lewej jest więcej haplotypów?
</div>

## Jednoczesne genotypowanie wielu próbek przy wykorzystaniu gVCF

Istnieje możliwość jednoczesnego genotypowania wielu probek w normalnym trybie pracy działania HaplotypeCallera (joint calling). Jednakże nie jest to rozwiązanie skalowalne. Dlatego rozdzielono przetwarzanie plików BAM od właściwego genotypowania poprzez wprowadzenie tzw. plików gVCF. 

W trybie potoku GVCF HaplotypeCaller urachmiany jest z opcją -ERC GVCF dla każdego pliku BAM, co w wyniku generuje pliki z rozszerzeniem .gvcf posiadające poza informacją o wariantach dodatkowe dane o jakości pozostałych regionów. Następnie jednoczesne genotypowanie wielu próbek odbywa się poprzez wykonanie metody GenotypeGVCF na wielu plikach gvcf. 

Utwórzmy plik g.vcf dla jednego pliku bam (mother.bam).

In [ ]:
! gatk HaplotypeCaller \
    -R ${WORKSPACE}/ref/ref.fasta \
    -I ${WORKSPACE}/bam/mother.bam \
    -O ${WORKSPACE}/sandbox/motherHC.g.vcf \
    -ERC GVCF \
    -L 20:10,000,000-10,200,000

In [ ]:
igv_browser= igv_notebook.Browser({
    "genome": "hg19",
    "locus": "chr20:10002294-10002623",
    "tracks": [{
        "name": "mother HC debug BAM",
        "url": edugen_bucket + "bam/motherHCdebug.bam",
        "indexURL": edugen_bucket + "bam/motherHCdebug.bai",
        "format": "bam",
        "type": "alignment",
        "showSoftClips": "True",
        "colorBy": "tag",
        "colorByTag": "HC",
        "sortByTag": "RG"
        },
        {
        "name": "mother HC VCF",
        "url": edugen_bucket + "motherHC.g.vcf.gz",
        "indexURL": edugen_bucket + "motherHC.g.vcf.gz.tbi",
        "format": "vcf",
        "type": "variant"
        }]
    }
)

Co sie zmienilo w stosunku do VCF?
Poza wariantami pojawily sie szare bloki reprezentujace stopien pewnosci, ze w danym regionie nie ma wariantu. 
Zmiana bloku oznacza zmiane pewnosci, ze w danym regionie nie ma wariantow. 


In [ ]:
# Podejrzyj kilka wierszy g.vcf
! zcat ${WORKSPACE}/sandbox/motherHC.g.vcf.gz |  grep 10002458

Jeżeli podejrzymy zawartość gVCF, to zobaczymy, że w kolumnie ALT pojawia się pozycja \<NON_REF\>.
Identyfikator PL w kolumnie format oznacza prawdopodobieństwo występowania kolejnych alleli, włączając \<NON_REF\>. 
***

### Import gvcf dla trio do specjalnej bazy danych (GenomicsDB)

In [ ]:
! mkdir sandbox_trio
! gatk GenomicsDBImport \
    -V ${WORKSPACE}/gvcf/mother.g.vcf.gz \
    -V ${WORKSPACE}/gvcf/father.g.vcf.gz \
    -V ${WORKSPACE}/gvcf/son.g.vcf.gz \
    --genomicsdb-workspace-path ${WORKSPACE}/sandbox_trio/trio \
    --intervals 20:10,000,000-10,200,000

In [ ]:
# z bazy mozemy wyciagnac polaczony gvcf
! gatk SelectVariants \
    -R ${WORKSPACE}/ref/ref.fasta \
    -V gendb://sandbox_trio/trio \
    -O ${WORKSPACE}/sandbox/trio_selectvariants.g.vcf

<b>ALE warianty na tym etapie nie mają wyliczonych genotypów</b>

In [ ]:
! cat ${WORKSPACE}/sandbox/trio_selectvariants.g.vcf | grep 10002458

Dlatego ostatnim krokiem jest genotyopwanie

In [ ]:
! gatk GenotypeGVCFs \
    -R ${WORKSPACE}/ref/ref.fasta \
    -V gendb://sandbox_trio/trio \
    -O ${WORKSPACE}/sandbox/trioGGVCF.vcf \
    -L 20:10,000,000-10,200,000

In [ ]:
! bgzip -c ${WORKSPACE}/sandbox/trioGGVCF.vcf > ${WORKSPACE}/sandbox/trioGGVCF.vcf.gz
! tabix ${WORKSPACE}/sandbox/trioGGVCF.vcf.gz

In [ ]:
igv_browser= igv_notebook.Browser(
    {
        "genome": "hg19",
        "locus": "chr20:10002294-10002623",
        "tracks": [{
            "name": "trio VCF",
            "url": edugen_bucket + "trioGGVCF.vcf.gz",
            "indexURL": edugen_bucket + "trioGGVCF.vcf.gz.tbi",
            "format": "vcf",
            "type": "variant"
        }]
    }
)

***
# Poprawa genotypowania
Posiadając informację o rodzinie, lub też wykorzystując informacje o znanych genotypach w populacji możemy wyliczyć prawdopodobieństwo a posteriori za pomoca funkcji [CalculateGenotypePosteriors](https://gatk.broadinstitute.org/hc/en-us/articles/360037226592-CalculateGenotypePosteriors) z pakietu GATK. 

<img src="png/genotype_refine.jpg">

Jeżeli w pliku gVCF mamy informację o próbkach z badania trio, to możemy za pomoca pliku z rodowodem ([w formacie .ped](https://gatk.broadinstitute.org/hc/en-us/articles/360035531972-PED-Pedigree-format)) spróbować poprawić pewnośc genotypowania.

In [ ]:
! gatk CalculateGenotypePosteriors \
    -V ${WORKSPACE}/sandbox/trioGGVCF.vcf \
    -ped ${WORKSPACE}/trio.ped \
    --skip-population-priors \
    -O ${WORKSPACE}/sandbox/trioCGP.vcf

***
Możemy również posiłkować się danymi z baz populacyjnych, jak np. baza gnomAD, w tym celu należy jako argument podać plik VCF z częstościami oraz genotypami

In [ ]:
! gatk CalculateGenotypePosteriors \
    -V ${WORKSPACE}/sandbox/trioGGVCF.vcf \
    -ped ${WORKSPACE}/trio.ped \
    --supporting-callsets ${WORKSPACE}/resources/af-only-gnomad.chr20subset.b37.vcf.gz \
    -O ${WORKSPACE}/sandbox/trioCGP_gnomad.vcf

In [ ]:
! bgzip -c  ${WORKSPACE}/sandbox/trioGGVCF.vcf > ${WORKSPACE}/sandbox/trioGGVCF.vcf.gz
! tabix -f ${WORKSPACE}/sandbox/trioGGVCF.vcf.gz
! bgzip -c  ${WORKSPACE}/sandbox/trioCGP.vcf > ${WORKSPACE}/sandbox/trioCGP.vcf.gz
! tabix -f ${WORKSPACE}/sandbox/trioCGP.vcf.gz
! bgzip -c  ${WORKSPACE}/sandbox/trioCGP_gnomad.vcf > ${WORKSPACE}/sandbox/trioCGP_gnomad.vcf.gz
! tabix -f ${WORKSPACE}/sandbox/trioCGP_gnomad.vcf.gz

***
W IGV możemy zobaczyć czy poprawa genotypowania przyniosła skutki

In [ ]:
igv_browser= igv_notebook.Browser({
    "genome": "hg19",
    "locus": "chr20:10002294-10002623",
    "tracks": [{
        "name": "trio GVCF",
        "url": edugen_bucket + "trioGGVCF.vcf.gz",
        "indexURL": edugen_bucket + "trioGGVCF.vcf.gz.tbi",
        "format": "vcf",
        "type": "variant"
        },
        {
        "name": "trio CGP",
        "url": edugen_bucket + "trioCGP.vcf.gz",
        "indexURL": edugen_bucket + "trioCGP.vcf.gz.tbi",
        "format": "vcf",
        "type": "variant"
        },
        {
        "name": "trio CGP gnomad",
        "url": edugen_bucket + "trioCGP_gnomad.vcf.gz",
        "indexURL": edugen_bucket + "trioCGP_gnomad.vcf.gz.tbi",
        "format": "vcf",
        "type": "variant"
        }]
    }
)

***
CalculateGenotypePosteriors dodaje 3 pozycje do kolumny FORMAT: PP, JL i JP.

- Phred-scaled Posterior Probability (PP) basically refines the PL values. It incorporates the prior expectations for the given pedigree and/or population allele frequencies.
- Joint Trio Likelihood (JL) is the Phred-scaled joint likelihood of the posterior genotypes for the trio being incorrect.
- Joint Trio Posterior (JP) is the Phred-scaled posterior probability of the posterior genotypes for the three samples being incorrect.

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://storage.googleapis.com/gatk-tutorials/workshop_1906/2-germline/images/vd-image4.png")

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://storage.googleapis.com/gatk-tutorials/workshop_1906/2-germline/images/vd-image5.png")

***
Więcej szczegółów na temat kolejnych kroków które można podjąć w celu polepszenia genotypowania: https://gatk.broadinstitute.org/hc/en-us/articles/360035531432?id=11074

# Analiza pokrycia

In [ ]:
import os
import pysam
counter = 0
with pysam.AlignmentFile(workspace+'/bam/mother.bam', 'rb') as samfile:
    for pileupcolumn in samfile.pileup("20", 10002294, 10002303):
        counter +=1;
        #if (pileupcolumn.pos != 16099988):   continue;
        print ("\ncoverage at base %s = %s" % (pileupcolumn.pos, pileupcolumn.n))
        if (counter > 10): break

In [ ]:
! samtools depth -a -r 20:10002294-10002623   ${WORKSPACE}/bam/mother.bam -o ${WORKSPACE}/sandbox/mother_cov

In [ ]:
! head  ${WORKSPACE}/sandbox/mother_cov

In [ ]:
import pandas as pd
tsv_read = pd.read_csv(workspace+'/sandbox/mother_cov', sep='\t' , header=None)

In [ ]:
tsv_read.columns = ["chr", "start", "coverage"]
tsv_read

<div class="alert alert-block alert-warning">
<b>Zadanie 9*:</b> Zrób wykres głębokości pokrycia dla pozycji (chr20:10002294-10002623)i porównaj go z wizualizacja w IGV.
</div>

# Adnotacje

In [ ]:
from biothings_client import get_client
import pandas as pd
pd.set_option('display.max_columns', 2000)
mv = get_client('variant')
mv.getvariants(["chr7:g.140453134T>C","chr7:g.140453134T>C"], as_dataframe=1)

In [ ]:
list(mv.query('dbnsfp.genename:CDK2', fetch_all=True))[0]
#http://docs.myvariant.info/en/latest/doc/data.html#available-fields

In [ ]:
#mv.getvariants('chr17:g.7578532A>G', fields = ['clinvar','gnomad_af'] , as_dataframe=1)
gg = mv.getvariants('chr7:g.140453134T>C', fields = "all" , as_dataframe=1)
gg.shape
gg

#https://cdn.rawgit.com/biothings/myvariant.info/master/docs/ipynb/myvariant_clinvar_demo.html

In [ ]:
#! gunzip -k ${WORKSPACE}/sandbox/motherHC.vcf
df = mv.getvariants(mv.get_hgvs_from_vcf(workspace + "/sandbox/motherHC.vcf") \
                    ,fields="all", as_dataframe=1, fetch_all=True)


In [ ]:
df[1:10]

In [ ]:
pd.options.display.max_seq_items = 2000
df.columns

In [ ]:
df["gnomad_genome.af.af"].plot(figsize=(20,10))

In [ ]:
df.groupby(['dbsnp.gene.name']).count()